<a href="https://colab.research.google.com/github/camlischke1/UnsupervisedLearningMNIST/blob/master/mnist_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Multi-class classification via unsupervised methods
This Jupyter notebook implements Hierarchical Agglomerative Clustering and KMeans clustering to cluster the MNIST dataset into 10 clusters. 
### Multi-class classification: the MNIST dataset
The MNIST dataset consists of 70,000 gray-scale images (samples) of hand-written digits 0 through 9. The multi-class classification problem consists of classifying each sample accurately as belonging to one of ten classes. For the purpose of this assignment, we will only use the labels provided for each sample in the dataset for determining classification accuracy of our results. 

In [ ]:
import numpy as np
import pandas as pd
import scipy
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb
import sklearn 
from sklearn.cluster import AgglomerativeClustering
import sklearn.metrics as sm
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.utils import to_categorical

def mapCluster(y_clusters, y, numClusters):
    """
    Parameters:
        y, a 1-D Numpy array containing true class numbers (integers between 0 and 10)
        y_cluster, a 1-D Numpy array containing cluster numbers corresponding to each element of y
        
    Returns:
        y_pred, a 1-D Numpy array mapping clusters to most common class in y 
    """
    y_pred = np.zeros(y_clusters.shape).astype(np.int32)
    mapOfCluster = np.zeros(numClusters).astype(np.int32) * -1
    # Map clusters to digits in mapOfClusters by choosing the digit most often associated with each cluster
    for cluster in range(0, numClusters):
        # Count number of true labels for each member in this cluster
        count = np.bincount(y[y_clusters == cluster], minlength=10)
        # Map cluster to the true label of most cluster members
        # Pick the index of the first max valuein count if there are more than max values
        mapOfCluster[cluster] = np.argmax(count)
    
        # print(cluster, "map to", y_pred[cluster], ", counts: ", count)

    # Now use mapOfCluster to map each y_cluster to predicted class
    y_pred = mapOfCluster[y_clusters]
    # This is the loop version of the line above
    # for i in range(0, len(y_clusters)):
    #     y_pred[i] = mapOfCluster[y_clusters[i]]
    
    return y_pred

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#setting printing standards and plot settings
np.set_printoptions(precision=4, suppress=True)
plt.figure(figsize=(10,3))
%matplotlib inline
plt.style.use('seaborn-whitegrid')

#import dataset
(train_images , train_labels), (test_images ,test_labels) = mnist.load_data()
#PREPROCESSING
#data needs to be in format that the NN expects as input
#before, it was uint8 array of shape (60000, 28, 28) but now we make it a float32 array with [0-1) float values
train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype('float32')/255

#renaming for simplicity
X = train_images[:1000]
Y = train_labels[:1000] #don't use these for the classification process
'''
Z = linkage(X, 'ward')
dendrogram(Z, truncate_mode='lastp', p=12, leaf_rotation = 45., leaf_font_size=15., show_contracted=True)
plt.title("Truncated Hierarchical Clustering Dendrogram")
plt.xlabel('Cluster Size')
plt.ylabel('Distance')
plt.show()'''

#generating hierarchical clustering and testing parameters
k=10      #because digits can be 0-9

Hclustering = AgglomerativeClustering(n_clusters = k, affinity='euclidean', linkage = 'ward')
Hclustering.fit(X)
'''
matrix = np.zeros((10,10))
for i in range(len(Hclustering.labels_)):
    matrix[Hclustering.labels_[i]][Y[i]] = matrix[Hclustering.labels_[i]][Y[i]]+1
#print(matrix)'''
#WE NEED TO MAP THE CLUSTERS TO THEIR RESPECTIVE TRUTH LABELS HERE (cluster, maplabel)
#relabel = np.choose(Hclustering.labels_,[0,4,8,1,3,6,2,7,5,9])
relabel = mapCluster(Hclustering.labels_, Y, k)
print(sm.accuracy_score(Y, relabel))
#print(sm.confusion_matrix(Y, relabel))


Hclustering = AgglomerativeClustering(n_clusters = k, affinity='euclidean', linkage = 'complete')
Hclustering.fit(X)
relabel = mapCluster(Hclustering.labels_, Y, k)
print(sm.accuracy_score(Y, relabel))
#print(sm.confusion_matrix(Y, relabel))

Hclustering = AgglomerativeClustering(n_clusters = k, affinity='euclidean', linkage = 'average')
Hclustering.fit(X)
relabel = mapCluster(Hclustering.labels_, Y, k)
print(sm.accuracy_score(Y, relabel))
#print(sm.confusion_matrix(Y, relabel))

Hclustering = AgglomerativeClustering(n_clusters = k, affinity='manhattan', linkage = 'complete')
Hclustering.fit(X)
relabel = mapCluster(Hclustering.labels_, Y, k)
print(sm.accuracy_score(Y, relabel))
#print(sm.confusion_matrix(Y, relabel))

Hclustering = AgglomerativeClustering(n_clusters = k, affinity='manhattan', linkage = 'average')
Hclustering.fit(X)
relabel = mapCluster(Hclustering.labels_, Y, k)
print(sm.accuracy_score(Y, relabel))
#print(sm.confusion_matrix(Y, relabel))

Hclustering = AgglomerativeClustering(n_clusters = k, affinity='cosine', linkage = 'average')
Hclustering.fit(X)
relabel = mapCluster(Hclustering.labels_, Y, k)
print(sm.accuracy_score(Y, relabel))
#print(sm.confusion_matrix(Y, relabel))

Hclustering = AgglomerativeClustering(n_clusters = k, affinity='cosine', linkage = 'complete')
Hclustering.fit(X)
relabel = mapCluster(Hclustering.labels_, Y, k)
print(sm.accuracy_score(Y, relabel))
#print(sm.confusion_matrix(Y, relabel))




11493376/11490434 [==============================] - 0s 0us/step
0.571
0.396
0.279
0.441
0.28
0.246
0.423


<BR>
### BACKGROUND OF HIERARCHICAL CLUSTERING

Hierarchical clustering, also known as agglomerative clustering, is an unsupervised machine learning method that takes a dataset, calculates some distace given a distance function, and maps each datapoint to its nearest neighbors. The best way I can explain this is by looking at the dendrogram above. The scikit learn AgglomerativeClustering method above takes in the mnist dataset. Based on the parameters chosen for the affinity and linkage parameters, the functions calculates the distance between these data points. It is a bottom-up clustering method that plots these points and groups the points together that are closest. As a datapoint is processed, it is added to the cluster that it relates to closest. The dendrogram perfectly depicts this, as it clusters at different levels. The root of the tree is the unique cluster that gathers all the samples, the leaves being the clusters with only one sample. As a user, you must either know how many classification/clusters you want, or know the maximum distance that you want to use. Hierarchical clustering does not require us to specify the number of clusters and we can even select which number of clusters looks best since we are building a tree.  If you know the maximum distance, you can easily discern the number of clusters at that distance by drawing a horizontal line on the dendrgram at that distance. Whatever clusters intercept that line, that is a cluster you will be using. 

To begin, each datapoint is its own cluster. BAsed on the linkage parameter, clusters are joined together one by one, until there is only the root cluster left. With the Average linkage, the two clusters with the smallest average distance away from each other will be linked. Ward's linkage gets the sum of the square of the distances, and then links the two clusters with the smallest values. The Complete linkage similarity is the maximum similarity between any pair of points in clustera and clusterb. 

The distance parameter can be Euclidean, Manhattan, or Cosine functions.  Euclidean looks at the sum of the distance between two points on a cartesian plane. Manhattan distance is very similar, however, it accounts for obstacles that cannot allow for Euclidean distance to be calculated and is less sensitive to outliers. The cosine distance looks at the size of the angle between two vectors that are extended infinitely.

High space and time complexity for Hierarchical clustering. Hence this clustering algorithm cannot be used when we have huge data.

<BR>
### IMPLEMENTATION OF HIERARCHICAL CLUSTERING

I started by downloading the mnist dataset from the keras library into the four numpy arrays. I only used the train_images half of the dataset, for simplicity. I reshaped the tensor from a 3D array of shape (60000,28,28) to a 2D tensor of (60000,28*28). This means there are 60,000 samples of an array of size 28x28. Then I stored each pixel as a float from 0 to 1. To do this, I divided each pixel's stored value by 255. I drew the dendrogram just to give a visualization. I then set X to the first 1000 elements of the train_images and Y to the first 1000 labels in the train_labels. I kept it at this low of a batch just to ensure that it ran without errors quickly. I knew that the number of clusters needed to be 10 due to the classification of our ten digits. I ran AgglomerativeClustering method with each pair of possible parameters, with n_clusters=10 for every method. This returned a new labeling known as Hclustering.labels_. However, this new labelling does not map directly to our digit classification. It can only describe the clusters returned by the function. I used Dr. Pauca's method to map the labels to our truth labels held in Y and printed the accuracy score of our Hierarchical clustering algorithm on this cluster. I then repeated this process for all 7 parameter pairings on all datapoints in batches of 10,000 datapoints at a time. 

<BR>
### RESULTS FOR HIERARCHICAL CLUSTERING

Average for Euclidian and Ward parameters: 65.67%
The confusion matrices AND ACCURACY SCORES can all be seen at this document:[ClusteringResults](https://drive.google.com/file/d/16SllTAyZF6QctNoCn16th-RBYCGQ097I/view?usp=sharing)

<BR>
### ANALYSIS OF HIERARCHICAL CLUSTERING

Foremost, it is important to note that clustering is not a classification tool. The method that Dr. Pauca supplied to map our clusters to classifications helps, but it is not as solid as some other options.

To me, it is not a shocak that the Euclidian and Ward parameters were the most accurate. The Euclidean distance formula looks at the sum of the distances of the vectors and the Ward linkage method gets the sum of the square of the distances, and then links the two clusters with the smallest values. These two parameters tend to produce more compact clusters. But with Ward's method, the clusters are usually equal in size and it extremely sensitive to outliers. 

As you can see in the results, the accuracy is not fantastic. I think this has a lot to do with the fact that the 28*28 image could have handwriting in the top right corner or the bottom left corner. The order and sequence of the pixels in the vector with higher intensity values is extremely varied in that case. Even though they could be representing the same digit, the location within the 28x28 image could skew the datat. In addition, it is an unsupervised method, so if the machine makes a mistake early on in the process, there is no way to correct it, and that mistake could cluster the remaining datapoints inaccurately. 





In [ ]:
#K-means clustering here
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale
import sklearn.metrics as sm
from sklearn import datasets
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.cluster import MiniBatchKMeans

from keras.datasets import mnist
from keras import models
from keras import layers
from keras.utils import to_categorical


def mapCluster(y_clusters, y, numClusters):
    """
    Parameters:
        y, a 1-D Numpy array containing true class numbers (integers between 0 and 10)
        y_cluster, a 1-D Numpy array containing cluster numbers corresponding to each element of y
        
    Returns:
        y_pred, a 1-D Numpy array mapping clusters to most common class in y 
    """
    y_pred = np.zeros(y_clusters.shape).astype(np.int32)
    mapOfCluster = np.zeros(numClusters).astype(np.int32) * -1
    # Map clusters to digits in mapOfClusters by choosing the digit most often associated with each cluster
    for cluster in range(0, numClusters):
        # Count number of true labels for each member in this cluster
        count = np.bincount(y[y_clusters == cluster], minlength=10)
        # Map cluster to the true label of most cluster members
        # Pick the index of the first max valuein count if there are more than max values
        mapOfCluster[cluster] = np.argmax(count)
    
        # print(cluster, "map to", y_pred[cluster], ", counts: ", count)

    # Now use mapOfCluster to map each y_cluster to predicted class
    y_pred = mapOfCluster[y_clusters]
    # This is the loop version of the line above
    # for i in range(0, len(y_clusters)):
    #     y_pred[i] = mapOfCluster[y_clusters[i]]
    
    return y_pred

#setting printing standards and plot settings
np.set_printoptions(precision=4, suppress=True)
plt.figure(figsize=(10,3))
%matplotlib inline
plt.style.use('seaborn-whitegrid')


#import dataset
(train_images , train_labels), (test_images ,test_labels) = mnist.load_data()
#PREPROCESSING
#data needs to be in format that the NN expects as input
#before, it was uint8 array of shape (60000, 28, 28) but now we make it a float32 array with [0-1) float values
train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype('float32')/255
X=train_images
Y=train_labels

%matplotlib inline
k = len(np.unique(test_labels))   #should be 10 clusters

kmeans = KMeans(n_clusters = k, random_state=1234)
kmeans.fit(X)

# kmeans.labels_ are the assigned clusters, NOT CLASSIFICATIONS
# from this eye-balling, I think cluster 2 -> 0 digit   cluster 9 -> 5
#                                        5 -> 8                 6 -> 7
#                                        1 -> 4                 8 -> 2
#                                        7 -> 9                 0 -> 1 
#                                        3 -> 6                 4 -> 3
'''
matrix = np.zeros((10,10))
for i in range(len(kmeans.labels_)):
    matrix[kmeans.labels_[i]][Y[i]] = matrix[kmeans.labels_[i]][Y[i]]+1
print(matrix)'''
#relabel = np.choose(kmeans.labels_,[9,1,4,2,3,5,0,7,6,8])
relabel = mapCluster(kmeans.labels_, Y, k)
print(sm.accuracy_score(Y, relabel))
print(sm.confusion_matrix(Y,relabel))

0.5910333333333333
[[5309   24   17  162   38    0  182   14  177    0]
 [   0 6695    9    5    6    0    8    9   10    0]
 [ 110  714 4194  328  173    0  212   69  158    0]
 [ 142  533  217 3926  175    0   55   48 1035    0]
 [  22  468   37    1 3188    0  165 1941   20    0]
 [ 337  968   14 1770  377    0  121  353 1481    0]
 [ 221  495   86   29   82    0 4914    1   90    0]
 [  29  610   39    5 1795    0    4 3773   10    0]
 [  70  718   54 1124  196    0   47  179 3463    0]
 [  56  357   13   84 2902    0    8 2460   69    0]]



<BR>
### BACKGROUND OF KMEANS CLUSTERING

KMeans Clustering clusters data points such that the sum of the square of the distances from each datapoint and the centroid of the cluster is kept at a minimum. Basically, it assigns clusters to keep its datapoints as close as possible, and keeps the clusters as far away as possible. I picture it as a scatterplot that has different colored datapoints based on the clusters each point belongs to. 

 The algorithm goes as follows. In order to use it, you must specify the number of clusters. You initialize the centroids by shuffling the dataset and randomly selecting K datapoints for the for the centroids. You can remove this randomization by choosing different random state parameters, like I did above. The next step is iterated until no datpoints are changed centroid assignments: compute the sum of the squared distances between all data points and all centroids, then assign each data point to the closest cluster, then compute the centroids for the clusters by taking the average of all data points. When this last step stops changing assignments, the algorithm is finished.

<BR>
###IMPLEMENTATION OF KMEANS CLUSTERING
I started by downloading the mnist dataset from the keras library into the four numpy arrays. I only used the train_images half of the dataset, for simplicity. I reshaped the tensor from a 3D array of shape (60000,28,28) to a 2D tensor of (60000,28*28). This means there are 60,000 samples of an array of size 28x28. Then I stored each pixel as a float from 0 to 1. To do this, I divided each pixel's stored value by 255. I then set X to the first 1000 elements of the train_images and Y to the first 1000 labels in the train_labels. I kept it at this low of a batch just to ensure that it ran without errors quickly. I knew that the number of clusters needed to be 10 due to the classification of our ten digits.

I initialized a variable called kmeans to a new KMeans object. The parameters were set to 10 n_clusters and the random state was 1234. In order to ensure that this method returns the same results on each execution, I set the random state to 1234. This is just an arbitrary number I chose to keep consistency. I fitted the kmeans object to the input data and it returned a new labeling of clusters. These labels are not maped to the truth labels that actually give us the digit number. I used Dr. Pauca's method to map, and printed the accuracy score, as well as the confusion matrix. 

<BR>
###RESULTS FOR KMEANS CLUSTERING

I was able to run the whole dataset in less than 4 minutes at an accuracy of .5910333. And the confusion matrix is also printed, and available here AT THE VERY BOTTOM: [ClusteringConfusionMatrices](https://drive.google.com/file/d/16SllTAyZF6QctNoCn16th-RBYCGQ097I/view) 

<BR>
###ANALYSIS OF KMEANS CLUSTERING

The accuracy for kmeans clustering is not fantastic. 59% is nowhere near the optimal solution. This could be due to a number of issues. Foremost, it is important to note that clustering is not a classification tool. The method that Dr. Pauca supplied to map our clusters to classifications helps, but it is not as solid as some other options.

The first of which has a lot to do with the fact that the 28*28 image could have handwriting in the top right corner or the bottom left corner. The order and sequence of the pixels in the vector with higher intensity values is extremely varied in that case. Even though they could be representing the same digit, the location within the 28x28 image could skew the datat. In addition, it is an unsupervised method, so if the machine makes a mistake early on in the process, there is no way to correct it, and that mistake could cluster the remaining datapoints inaccurately. 

Another possible variance is the random_state parameter used in the KMeans object. I ran the same code, but changed the parameter from '1234' to '123'. Upon this iteration, the accuracy score declined by .03 points. I would not be shocked if there were some random states that improved the accuracy scores for this dataset.  



<BR> 
### Comparison of Hierarchical Clustering to KMeans</BR>

Although both methods are unsupervised, there are a numbe rof differences between hierarchical agglomerative and kmeans clustering. The first of which that caught my attention is that with kmeans, you have to immediately know how many clusters you'd prefer in the dataset. With hierarchical clustering, all you need is a maximum distance and the dendrogram will give you the number of clusters that fulfill that distance. With hierarchical, you can either have a specified distance or a number of clusters, which adds to the flexibility. K Means clustering requires prior knowledge of K i.e. no. of clusters you want to divide your data into. But, you can stop at whatever number of clusters you find appropriate in hierarchical clustering by interpreting the dendrogram. 

In addition, there are efficiency differences. I did not try to use the 60,000 element sample size for hierarchical clustering because I know that it would take extremely long. Based on towardsdatascience.com, hierarchical clustering is O(n^3) complexity, whereas kmeans can be done in linear time. Kmeans' algorithm is to add a datapoint, sum the squared distance, and recalculate centroids for each dataset, which determines its linear complexity. Therefore, Kmeans might be better for large datasets. 

There is also a randomized aspect of kmeans when beginning the algorithm. Although I disengaged this ability for my purposes, hierarchical clustering is deterministic in nature. This makes kmeans a little harder to use as a beginner because it might skew your results if (a) your data is not well-separated into sphere-like clusters, (b) you pick a 'k' not well-suited to the shape of your data, i.e. you pick a value too high or too low, or (c) you have weird initial values for your cluster centroids.






